<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-20 16:22:03
-------------------
qualified stocks: 96
with latest results: 33
still star stocks: 19


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  55.89 K
Current:  1.27 C
-------------------
Today PnL: -1.45 L (-1.13%)
Current PnL: -16.88 L (-11.97%)
CY Booked + Current PnL: -13.72 L (-9.73%)
-------------------
Total profit:  4.21 L
Total loss:  -21.10 L
-------------------
Total Booked + Current PnL: 13.26 L (11.62%)
Total Booked PnL: 30.14 L (26.43%)
Curr Year Booked PnL: 3.16 L (2.49%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 79.98 L (63.0%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.41 C (111.42%)
Deployed:  1.14 C
Current:  1.27 C
CAGR/XIRR %: 7.68%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SBILIFE,1496.49,1924.99,-0.61,H-LC,72.92,147655.0,21950.0,14042.0,-0.69,17.46,9.51,28.63,159.0,1.56,1.17,31.54,AR,ATH,INSURANCE
63,RELIANCE,1288.53,1526.00,4.61,X-LC,79.17,225087.0,21499.0,16026.0,-1.12,10.56,7.12,18.43,161.0,1.34,1.78,22.61,XY25,NTT,REFINERIES
6,ANGELONE,2328.67,3033.00,10.77,H-SC,90.62,180505.0,29141.0,16643.0,-0.90,19.25,9.22,30.25,149.0,1.75,1.43,42.86,X40N,NTT,FINANCE
57,PIDILITIND,2504.06,3576.00,-18.67,X-LC,33.33,90084.0,14962.0,17197.0,-2.13,19.92,19.09,42.81,121.0,0.87,0.71,12.98,X40,BTT,CHEMICALS
71,SBIN,760.30,863.00,-15.33,M-LC,22.92,208197.0,6718.0,20507.0,-1.18,3.33,9.85,13.51,60.0,0.33,1.65,16.35,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PFIZER,4236.33,6318.30,-1.45,H-SC,97.92,153444.0,22118.0,42427.0,10.78,16.84,27.65,49.15,94.0,0.52,1.21,26.47,X40,ATH,PHARMA
21,COFFEEDAY,59.14,80.00,-57.75,L-SC,92.71,61574.0,-51975.0,92022.0,4.97,-45.77,149.45,35.27,181.0,-0.56,0.49,50.70,XR,NTT,HOTELS
86,TTKPRESTIG,769.29,1149.24,-1.25,X-SC,61.46,90246.0,-10531.0,60302.0,2.61,-10.45,66.82,49.39,172.0,-0.17,0.71,17.44,X40N,ATH,DURABLES
25,DLF,685.63,1031.70,-18.13,H-LC,86.46,169538.0,15271.0,62593.0,2.15,9.90,36.92,50.47,13.0,0.24,1.34,23.92,AR,ATH,REALTY
30,GREENPANEL,375.16,537.00,175.84,M-SC,60.42,123010.0,-60068.0,139051.0,1.83,-32.81,113.04,43.14,109.0,-0.43,0.97,15.71,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,QUESS,711.66,986.00,NaN,NaN,20.83,74099.0,-81755.0,141833.0,-9.35,-52.46,191.41,38.55,12.0,-0.58,0.59,13.46,XY24,NTT,MISC
84,TITAGARH,1097.23,1548.00,-11.39,H-SC,89.58,155522.0,-36493.0,115382.0,-5.16,-19.01,74.19,41.08,119.0,-0.32,1.23,31.63,XY24,NTT,ENGINEERING
27,EASEMYTRIP,18.12,26.40,16.70,M-SC,5.21,112412.0,-64403.0,145203.0,-4.71,-36.42,129.17,45.70,22.0,-0.44,0.89,4.35,XY24,NTT,TRAVEL
92,VIPIND,488.80,718.00,-845.76,H-SC,100.00,71048.0,-24268.0,68959.0,-3.92,-25.46,97.06,46.89,173.0,-0.35,0.56,40.13,X40N,NTT,MISC
34,HEROMOTOCO,4311.81,6039.03,-2.86,H-MC,75.00,148456.0,-2457.0,62916.0,-3.33,-1.63,42.38,40.06,27.0,-0.04,1.17,21.43,AR,ATH,AUTO


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIAMART,2311.97,4911.36,-51.84,H-SC,58.33,103998.0,2271.0,112099.0,-0.83,2.23,107.79,112.43,1.0,0.02,0.82,22.74,AR,ATH,MISC
83,TCS,3928.29,4998.00,-16.16,X-LC,59.38,188908.0,-23220.0,80985.0,-0.58,-10.95,42.87,27.23,3.0,-0.29,1.49,8.26,X40,BTT,IT
82,TATAMOTORS,776.02,1065.00,-52.64,M-LC,91.67,190508.0,-15137.0,91711.0,-1.45,-7.36,48.14,37.24,4.0,-0.17,1.51,24.00,XY24,NTT,AUTO
10,AWL,342.88,485.00,-69.27,H-MC,2.08,166592.0,-51137.0,141387.0,-2.49,-23.49,84.87,41.45,4.0,-0.36,1.32,11.28,XY24,NTT,FMCG
42,INFY,1432.81,2275.00,-12.78,X-LC,68.75,216923.0,17762.0,99307.0,0.05,8.92,45.78,58.78,6.0,0.18,1.72,13.25,X40,BTT,IT


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,POLYCAB,5005.92,7541.35,7.69,H-MC,84.38,168378.0,28212.0,42785.0,-1.46,20.13,25.41,50.65,56.0,0.66,1.33,28.51,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-1.85,M-LC,85.42,189690.0,38484.0,27183.0,-0.98,25.45,14.33,43.43,48.0,1.42,1.50,31.48,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LUXIND,1448.19,2442.00,43.16,M-SC,66.67,148722.0,11144.0,83269.0,-1.78,8.10,55.99,68.62,66.0,0.13,1.18,28.93,X40N,NTT,TEXTILES
6,ANGELONE,2328.67,3033.00,10.77,H-SC,90.62,180505.0,29141.0,16643.0,-0.90,19.25,9.22,30.25,149.0,1.75,1.43,42.86,X40N,NTT,FINANCE
20,COALINDIA,406.65,512.15,8.40,L-LC,45.83,150997.0,536.0,38504.0,1.34,0.36,25.50,25.94,85.0,0.01,1.19,15.22,XY25,ATH,MINING
58,POLYCAB,5005.92,7541.35,7.69,H-MC,84.38,168378.0,28212.0,42785.0,-1.46,20.13,25.41,50.65,56.0,0.66,1.33,28.51,X40N,ATH,CABLES
29,GILLETTE,8109.44,10330.69,0.27,H-SC,81.25,147960.0,10100.0,27669.0,0.86,7.33,18.70,27.39,64.0,0.37,1.17,29.52,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-3.78,H-SC,16.67,108276.0,6272.0,38752.0,-2.14,6.15,35.79,44.14,31.0,0.16,0.86,23.39,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-33.09,M-MC,17.71,132201.0,3049.0,84014.0,-1.50,2.36,63.55,67.42,57.0,0.04,1.05,28.58,XR,NTT,BANKS
95,WIPRO,243.46,420.00,-9.71,M-LC,57.29,154820.0,3875.0,105587.0,-0.81,2.57,68.20,72.51,49.0,0.04,1.22,16.57,XR,NTT,IT
39,INDIAMART,2311.97,4911.36,-51.84,H-SC,58.33,103998.0,2271.0,112099.0,-0.83,2.23,107.79,112.43,1.0,0.02,0.82,22.74,AR,ATH,MISC
1,AARTIIND,487.04,919.00,37.28,M-SC,88.54,78857.0,-2479.0,74614.0,-2.99,-3.05,94.62,88.69,169.0,-0.03,0.62,32.98,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VALIANTORG,512.64,838.00,-342.67,H-SC,71.88,103285.0,-30001.0,114595.0,0.98,-22.51,110.95,63.47,145.0,-0.26,0.82,73.13,XR,NTT,CHEMICALS
87,UJJIVANSFB,52.77,60.00,38.86,M-SC,39.58,120312.0,-22167.0,41688.0,-0.85,-15.56,34.65,13.70,213.0,-0.53,0.95,42.00,X40N,NTT,BANKS
12,BANKINDIA,113.49,190.00,-33.09,M-MC,17.71,132201.0,3049.0,84014.0,-1.50,2.36,63.55,67.42,57.0,0.04,1.05,28.58,XR,NTT,BANKS
70,SBILIFE,1496.49,1924.99,-0.61,H-LC,72.92,147655.0,21950.0,14042.0,-0.69,17.46,9.51,28.63,159.0,1.56,1.17,31.54,AR,ATH,INSURANCE
88,UNIONBANK,123.87,163.00,-9.51,M-LC,70.83,157406.0,16566.0,27924.0,1.07,11.76,17.74,31.59,70.0,0.59,1.25,36.73,XY24,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,718.00,-845.76,H-SC,100.00,71048.0,-24268.0,68959.0,-3.92,-25.46,97.06,46.89,173.0,-0.35,0.56,40.13,X40N,NTT,MISC
94,WHIRLPOOL,1151.42,2270.00,-34.66,M-SC,98.96,164528.0,18298.0,123758.0,0.15,12.51,75.22,97.15,28.0,0.15,1.30,40.56,X40,NTT,DURABLES
56,PFIZER,4236.33,6318.30,-1.45,H-SC,97.92,153444.0,22118.0,42427.0,10.78,16.84,27.65,49.15,94.0,0.52,1.21,26.47,X40,ATH,PHARMA
51,LTIM,5664.39,7262.39,-6.27,X-LC,96.88,150462.0,-19470.0,67407.0,-0.09,-11.46,44.80,28.21,32.0,-0.29,1.19,23.62,X200,ATH,IT
80,TANLA,917.30,1963.11,-40.21,H-SC,95.83,146760.0,-92655.0,365609.0,-3.25,-38.70,249.12,114.01,20.0,-0.25,1.16,34.60,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.01
1,25,38.33
2,50,67.68


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.42
X40N    15.27
X40     14.30
AR      13.94
XR      10.39
XY25     9.39
X200     1.19
SR       1.10
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.65
M-SC    17.66
X-LC    15.92
H-MC     8.85
M-LC     8.28
H-LC     5.91
M-MC     5.53
X-MC     5.51
X-SC     2.81
L-SC     2.29
L-LC     1.19
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.51,-2.03,34.49
BANKS,10.87,-9.62,63.39
IT,9.06,-20.78,88.17
FINANCE,7.45,-17.60,70.67
MISC,7.08,-26.51,80.94
INSURANCE,6.13,0.26,29.81
PAINTS,5.50,-18.26,53.26
AUTO,4.30,-11.45,60.03
ELECTRICAL,4.06,-8.29,63.47


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3004059.0,27
AR,1299026.0,15
XR,1231848.0,15
X40N,1231805.0,18
X40,785606.0,12
SR,205832.0,2
XY25,172821.0,6
X200,67407.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2324880.0,22
M-SC,2004242.0,21
X-LC,728760.0,12
M-MC,570271.0,5
H-MC,489301.0,8
X-SC,353217.0,4
M-LC,334619.0,6
X-MC,308887.0,5
L-SC,307782.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      908093.0      6
M-SC       XY24      759661.0      6
H-SC       AR        537152.0      3
           X40N      527017.0      7
M-SC       X40N      444701.0      6
           XR        425053.0      5
M-MC       XY24      401986.0      3
X-LC       X40       390410.0      5
H-SC       XR        282522.0      4
M-LC       XY24      208525.0      4
M-SC       X40       196804.0      2
H-LC       AR        195397.0      4
X-LC       AR        186765.0      3
H-MC       XY24      180345.0      2
L-SC       XR        157580.0      2
X-MC       XY24      146125.0      2
H-MC       AR        138800.0      2
X-SC       SR        120660.0      1
           XR        119079.0      1
           X40N      113478.0      2
M-LC       XR        105587.0      1
X-MC       X40N      103824.0      2
M-SC       AR         92851.0      1
L-SC       XY24       86412.0      1
M-SC       SR         85172.0      1
M-MC       AR         84271.0      1
X-LC       XY25       84178.0      3
M-MC       XR         84014.0      1
H-SC       X40        70096.0      2
H-MC       X40        69358.0      2
X-LC       X200       67407.0      1
L-SC       AR         63790.0      1
X-MC       X40        58938.0      1
H-MC       XR         58013.0      1
           X40N       42785.0      1
L-LC       XY25       38504.0      1
H-LC       XY25       29632.0      1
M-LC       XY25       20507.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
